In [ ]:
import gymnasium as gym


env = gym.make("FrozenLake-v1", map_name="8x8", is_slippery=False, render_mode="human")

observation, info = env.reset()

for _ in range(100):
    action = env.action_space.sample()
    print(f"Action: {action}")
    observation, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
        observation, info = env.reset()
        print("Environment reset")

env.close()

Action/Ob Space Inspection

In [75]:
env = gym.make("FrozenLake-v1", map_name="4x4", is_slippery=True, render_mode="rgb_array")

env.reset()
print("_____OBSERVATION SPACE_____ \n")
print("Observation Space Shape", env.observation_space.shape)
print("Sample observation", env.observation_space.sample())  # Get a random observation

print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space.n)
print("Action Space Sample", env.action_space.sample())  # Take a random action

_____OBSERVATION SPACE_____ 

Observation Space Shape ()
Sample observation 10

 _____ACTION SPACE_____ 

Action Space Shape 4
Action Space Sample 3


Model

In [48]:
import gymnasium as gym
import numpy as np

In [76]:
desc=["SFFF", 
      "FHFH", 
      "FHFH", 
      "HFFG"]

env = gym.make("FrozenLake-v1", map_name="4x4", is_slippery=True, max_episode_steps=1000, render_mode="rgb_array")
env.reset()

state_space = env.observation_space.n
action_space = env.action_space.n

def initialize_q_table(state_space, action_space):
    return np.zeros((state_space, action_space))

q_table = initialize_q_table(state_space, action_space)


In [77]:
# policies
def greedy_policy(q_table, state):
    return np.argmax(q_table[state, :])

def epsilon_greedy_policy(q_table, state, epsilon):
    random_num = np.random.random()
    if random_num > epsilon:
        action = greedy_policy(q_table, state)
    else:
        action = env.action_space.sample()
    return action

In [78]:
# hyperparameters
n_training_episodes = 100000
learning_rate = 0.7

n_eval_episodes = 100

env_id = "FrozenLake-v1"
max_steps = 999
gamma = 0.95
eval_seed = []

max_epsilon = 1.0
min_epsilon = 0.05
epsilon_decay = 0.00005

In [79]:
def train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable):
    for episode in range(n_training_episodes):
        epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate * episode)
        print(epsilon)
        state, info = env.reset()
        step = 0
        terminated = False
        truncated = False

        for step in range(max_steps):
            action = epsilon_greedy_policy(Qtable, state, epsilon)
            observation, reward, terminated, truncated, info = env.step(action)
            Qtable[state, action] = Qtable[state, action] + learning_rate * (reward + gamma * np.max(Qtable[observation, :]) - Qtable[state, action])
            if terminated or truncated:
                break
            state = observation
    return Qtable


In [80]:
q_table = train(n_training_episodes, min_epsilon, max_epsilon, epsilon_decay, env, max_steps, q_table)

1.0
0.9999525011874801
0.9999050047498417
0.9998575106869656
0.9998100189987335
0.9997625296850262
0.9997150427457253
0.9996675581807121
0.9996200759898677
0.9995725961730736
0.9995251187302109
0.9994776436611609
0.9994301709658051
0.9993827006440248
0.9993352326957011
0.9992877671207157
0.9992403039189496
0.9991928430902841
0.999145384634601
0.9990979285517811
0.9990504748417063
0.9990030235042575
0.9989555745393163
0.998908127946764
0.9988606837264821
0.9988132418783519
0.9987658024022547
0.998718365298072
0.9986709305656853
0.998623498204976
0.9985760682158253
0.9985286405981149
0.9984812153517261
0.9984337924765402
0.9983863719724388
0.9983389538393035
0.9982915380770154
0.9982441246854562
0.9981967136645074
0.9981493050140502
0.9981018987339664
0.9980544948241373
0.9980070932844445
0.9979596941147694
0.9979122973149935
0.9978649028849984
0.9978175108246655
0.9977701211338764
0.9977227338125126
0.9976753488604557
0.9976279662775871
0.9975805860637885
0.9975332082189413
0.9974858327

In [81]:
q_table

array([[1.21592182e-01, 1.34538910e-01, 1.21756389e-01, 9.92807459e-02],
       [1.79990911e-02, 5.42389625e-02, 2.16280744e-02, 1.19688588e-01],
       [3.53743473e-02, 3.50908520e-02, 2.52564287e-02, 8.22045461e-02],
       [2.07883992e-02, 4.34543115e-03, 1.11860716e-02, 4.59501765e-02],
       [1.69210904e-01, 1.25396063e-01, 6.46301535e-02, 6.62066188e-03],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.27419157e-02, 6.69788291e-04, 2.32418544e-03, 2.07884430e-03],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.68383229e-02, 2.44390974e-02, 6.75826362e-02, 1.86857540e-01],
       [4.57018517e-02, 2.95701290e-01, 1.20088692e-01, 1.50885799e-01],
       [1.15331362e-01, 2.24823516e-02, 6.76494595e-02, 2.54932579e-02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.63667332e-01, 3.80742190e-01, 5.02476867e

Evaluate

In [82]:
def evaluate_agent(env, max_steps, n_eval_episodes, Q, seed):
  """
  Evaluate the agent for ``n_eval_episodes`` episodes and returns average reward and std of reward.
  :param env: The evaluation environment
  :param max_steps: Maximum number of steps per episode
  :param n_eval_episodes: Number of episode to evaluate the agent
  :param Q: The Q-table
  :param seed: The evaluation seed array (for taxi-v3)
  """
  episode_rewards = []
  for episode in range(n_eval_episodes):
    if seed:
      state, info = env.reset(seed=seed[episode])
    else:
      state, info = env.reset()
    step = 0
    truncated = False
    terminated = False
    total_rewards_ep = 0

    for step in range(max_steps):
      # Take the action (index) that have the maximum expected future reward given that state
      action = greedy_policy(Q, state)
      new_state, reward, terminated, truncated, info = env.step(action)
      total_rewards_ep += reward

      if terminated or truncated:
        break
      state = new_state
    episode_rewards.append(total_rewards_ep)
  mean_reward = np.mean(episode_rewards)
  std_reward = np.std(episode_rewards)

  return mean_reward, std_reward

In [83]:
# Evaluate our Agent
mean_reward, std_reward = evaluate_agent(env, max_steps, n_eval_episodes, q_table, eval_seed)
print(f"Mean_reward={mean_reward:.2f} +/- {std_reward:.2f}")

Mean_reward=0.44 +/- 0.50


Run Model

In [84]:
# Run the model for a few episodes
for episode in range(2):
    # Create a new environment for inference
    env = gym.make("FrozenLake-v1", map_name="4x4", is_slippery=True, render_mode="human")

    # Reset the environment
    obs, _ = env.reset()

    episode_reward = 0
    done = False
    truncated = False
    
    while not (done or truncated):
        # Get the model's action
        action = greedy_policy(q_table, obs)
        
        # Take the action in the environment
        obs, reward, done, truncated, info = env.step(action)
        episode_reward += reward
    
    print(f"Episode {episode + 1} reward: {episode_reward}")
    
    # Reset the environment for the next episode
    obs, _ = env.reset()

# Close the environment
env.close()


Episode 1 reward: 0.0
Episode 2 reward: 1.0
